# 🔽 Monta o Drive e utiliza o gdown para baixar o arquivo .mp4 diretamente de um link público do Google Drive.

In [1]:
## montar o google driver
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
DATA_DIR = "/content/drive/MyDrive/FASE 4 - Tech Challenge"

In [3]:
import gdown
import shutil
import os

zip_file_id = "1oKzEhA6jzU4YAUmSiN62_J_3-wztmjaC"
zip_output = DATA_DIR

# Use gdown para obter o nome original do arquivo
downloaded_path = gdown.download(
    f'https://drive.google.com/uc?id={zip_file_id}',
    quiet=False,
    output=None  # None mantém o nome original do arquivo do Drive
)

# Move o arquivo baixado para a pasta de destino mantendo o nome original
if downloaded_path is not None:
    nome_arquivo = os.path.basename(downloaded_path)
    caminho_destino = os.path.join(zip_output, nome_arquivo)
    shutil.move(downloaded_path, caminho_destino)
    print(f'Arquivo salvo em: {caminho_destino}')
else:
    print('Download falhou.')

Downloading...
From: https://drive.google.com/uc?id=1oKzEhA6jzU4YAUmSiN62_J_3-wztmjaC
To: /content/Unlocking Facial Recognition_ Diverse Activities Analysis.mp4
100%|██████████| 38.3M/38.3M [00:00<00:00, 51.3MB/s]


Arquivo salvo em: /content/drive/MyDrive/FASE 4 - Tech Challenge/Unlocking Facial Recognition_ Diverse Activities Analysis.mp4


In [4]:
!pip install deepface opencv-python moviepy tensorflow matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.0 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=34f4b03283ce52f9c2af79801b9a9fd04cae9287253df5a5966e7dfe7dc458a9
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


# Gera um novo vídeo com a marcação das faces e emoções identificadas

In [5]:
import cv2
import numpy as np
from deepface import DeepFace
from moviepy.editor import VideoFileClip, AudioFileClip

# Caminhos de entrada e saída
input_video_path = f"{DATA_DIR}/Unlocking Facial Recognition_ Diverse Activities Analysis.mp4"
output_video_path = f"{DATA_DIR}/output_video_audio_emotion.mp4"

# Inicializa captura de vídeo
cap = cv2.VideoCapture(input_video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define o codec e cria o VideoWriter (sem áudio)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
temp_video_path = f"{DATA_DIR}/temp_output_video_noaudio.mp4"
out = cv2.VideoWriter(temp_video_path, fourcc, fps, (width, height))

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"width: {width}")
print(f"height: {height}")
print(f"fps: {fps}")
print(f"total_frames: {frame_count}")

print("Processando vídeo, aguarde...")
frame_idx = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    try:
        # Detecta faces e analisa emoções
        faces = DeepFace.analyze(frame, actions=['emotion'], detector_backend='retinaface', enforce_detection=False)
        # DeepFace.analyze pode retornar um dict ou uma lista de dicts dependendo do número de faces
        if isinstance(faces, dict):
            faces = [faces]
        for face in faces:
            x, y, w, h = face['region']['x'], face['region']['y'], face['region']['w'], face['region']['h']
            emotion = face['dominant_emotion']
            # Retângulo ao redor da face
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            # Escreve a emoção acima do retângulo
            cv2.putText(frame, emotion, (x + 5, y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    except Exception as e:
        print(f"Erro na detecção em frame {frame_idx}: {e}")

    out.write(frame)
    frame_idx += 1
    if frame_idx % 30 == 0:
        print(f"Frame {frame_idx}/{frame_count}")

cap.release()
out.release()

# Usa moviepy para adicionar o áudio original ao vídeo processado
print("Adicionando áudio ao vídeo final...")
video_clip = VideoFileClip(temp_video_path)

audio_path = f"{DATA_DIR}/audio.wav"
audio_clip = AudioFileClip(input_video_path)
audio_clip.write_audiofile(audio_path)

final_clip = video_clip.set_audio(audio_clip)
final_clip.write_videofile(output_video_path, codec="libx264", audio_codec="aac")


25-06-22 21:43:52 - Directory /root/.deepface has been created
25-06-22 21:43:52 - Directory /root/.deepface/weights has been created
width: 1280
height: 720
fps: 30.0
total_frames: 3326
Processando vídeo, aguarde...
25-06-22 21:43:58 - retinaface.h5 will be downloaded from the url https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
To: /root/.deepface/weights/retinaface.h5
100%|██████████| 119M/119M [00:01<00:00, 117MB/s]


25-06-22 21:44:08 - facial_expression_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /root/.deepface/weights/facial_expression_model_weights.h5
100%|██████████| 5.98M/5.98M [00:00<00:00, 98.7MB/s]


Frame 30/3326
Frame 60/3326
Frame 90/3326
Frame 120/3326
Frame 150/3326
Frame 180/3326
Frame 210/3326
Frame 240/3326
Frame 270/3326
Frame 300/3326
Frame 330/3326
Frame 360/3326
Frame 390/3326
Frame 420/3326
Frame 450/3326
Frame 480/3326
Frame 510/3326
Frame 540/3326
Frame 570/3326
Frame 600/3326
Frame 630/3326
Frame 660/3326
Frame 690/3326
Frame 720/3326
Frame 750/3326
Frame 780/3326
Frame 810/3326
Frame 840/3326
Frame 870/3326
Frame 900/3326
Frame 930/3326
Frame 960/3326
Frame 990/3326
Frame 1020/3326
Frame 1050/3326
Frame 1080/3326
Frame 1110/3326
Frame 1140/3326
Frame 1170/3326
Frame 1200/3326
Frame 1230/3326
Frame 1260/3326
Frame 1290/3326
Frame 1320/3326
Frame 1350/3326
Frame 1380/3326
Frame 1410/3326
Frame 1440/3326
Frame 1470/3326
Frame 1500/3326
Frame 1530/3326
Frame 1560/3326
Frame 1590/3326
Frame 1620/3326
Frame 1650/3326
Frame 1680/3326
Frame 1710/3326
Frame 1740/3326
Frame 1770/3326
Frame 1800/3326
Frame 1830/3326
Frame 1860/3326
Frame 1890/3326
Frame 1920/3326
Frame 1950/3

MoviePy - Done.
Moviepy - Building video /content/drive/MyDrive/FASE 4 - Tech Challenge/output_video_audio_emotion.mp4.
MoviePy - Writing audio in output_video_audio_emotionTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/FASE 4 - Tech Challenge/output_video_audio_emotion.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/FASE 4 - Tech Challenge/output_video_audio_emotion.mp4


In [6]:
!pip install openai-whisper pysrt googletrans==4.0.0rc1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 16.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.3 MB/s et

# Gera as legendas em inglês e português

In [7]:
import whisper

legenda_en_path = f"{DATA_DIR}/legenda_en.srt"
model = whisper.load_model("base")
result = model.transcribe(audio_path, language="en", task="transcribe")
with open(legenda_en_path, "w", encoding="utf-8") as f:
    for segment in result["segments"]:
        # SRT format: index, time, text
        idx = segment["id"] + 1
        start = segment["start"]
        end = segment["end"]
        # Format time for SRT
        def format_time(t):
            h = int(t // 3600)
            m = int((t % 3600) // 60)
            s = int(t % 60)
            ms = int((t * 1000) % 1000)
            return f"{h:02}:{m:02}:{s:02},{ms:03}"
        f.write(f"{idx}\n{format_time(start)} --> {format_time(end)}\n{segment['text'].strip()}\n\n")

import pysrt
from googletrans import Translator

legenda_pt_path = f"{DATA_DIR}/legenda_pt.srt"

# Carregar legendas originais
subs = pysrt.open(legenda_en_path, encoding='utf-8')
translator = Translator()

# Traduzir cada linha da legenda
for sub in subs:
    translated = translator.translate(sub.text, src='en', dest='pt')  # traduz de inglês para português
    sub.text = translated.text

# Salvar legenda traduzida
subs.save(legenda_pt_path, encoding='utf-8')



100%|███████████████████████████████████████| 139M/139M [00:04<00:00, 35.9MiB/s]


# Inclui as legendas no vídeo

In [9]:
import subprocess

output_file = f"{DATA_DIR}/output_video_audio_emotion_com_legendas.mp4"

cmd = [
    "ffmpeg",
    "-i", output_video_path,
    "-vf",
    # Ajuste MarginV conforme o tamanho do vídeo
    f"subtitles={legenda_en_path}:force_style='FontName=Arial,FontSize=15,MarginV=10',subtitles={legenda_pt_path}:force_style='FontName=Arial,FontSize=12,MarginV=40,Outline=1,OutlineColour=&H00808080&'",
    "-c:a", "copy",
    output_file
]

subprocess.run(cmd)

CompletedProcess(args=['ffmpeg', '-i', '/content/drive/MyDrive/FASE 4 - Tech Challenge/output_video_audio_emotion.mp4', '-vf', "subtitles=/content/drive/MyDrive/FASE 4 - Tech Challenge/legenda_en.srt:force_style='FontName=Arial,FontSize=15,MarginV=10',subtitles=/content/drive/MyDrive/FASE 4 - Tech Challenge/legenda_pt.srt:force_style='FontName=Arial,FontSize=12,MarginV=40,Outline=1,OutlineColour=&H00808080&'", '-c:a', 'copy', '/content/drive/MyDrive/FASE 4 - Tech Challenge/output_video_audio_emotion_com_legendas.mp4'], returncode=0)